In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.examples.tutorials.mnist import input_data

In [25]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
tf.reset_default_graph()
# graph에 있는 모든 tensor 초기화!

x = tf.placeholder(tf.float32, [None, 28*28])
y = tf.placeholder(tf.float32, [None, 10])

w1 = tf.get_variable(name="w1", shape=[28*28, 16*16],\
                          initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([16*16]))
L1 = tf.nn.relu(tf.matmul(x,w1)+b1)

w2 = tf.get_variable(name="w2", shape=[16*16, 16*16],\
                          initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([16*16]))
L2 = tf.nn.relu(tf.matmul(L1,w2)+b2)

w3 = tf.get_variable(name="w3", shape=[16*16, 10],\
                          initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))

hf = tf.matmul(L2,w3)+b3
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hf, labels=y))

train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [14]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      myfeed = {x:batch_xs, y:batch_ys}
      cv, _ = sess.run([cost, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
    print("cost: {:.9f}".format(avg_cost))
    
  c_pre = tf.equal(tf.argmax(hf,1), tf.argmax(y,1))
  acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))
  print("acc: {}".format(sess.run(acc, feed_dict={x:mnist.test.images,y:mnist.test.labels})))

cost: 0.304808350
cost: 0.110608818
cost: 0.072338051
cost: 0.053155470
cost: 0.038621707
cost: 0.029877179
cost: 0.022208641
cost: 0.018917425
cost: 0.016415381
cost: 0.015246934
cost: 0.012983449
cost: 0.010256912
cost: 0.011718637
cost: 0.010002678
cost: 0.007746778
acc: 0.9799000024795532


# 이래저래 바꿔보자

In [0]:
tf.reset_default_graph()
# graph에 있는 모든 tensor 초기화!

x = tf.placeholder(tf.float32, [None, 28*28])
y = tf.placeholder(tf.float32, [None, 10])

w1 = tf.get_variable(name="w1", shape=[28*28, 512],\
                          initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(x,w1)+b1)

w2 = tf.get_variable(name="w2", shape=[512, 512],\
                          initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1,w2)+b2)

w3 = tf.get_variable(name="w3", shape=[512, 512],\
                          initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2,w3)+b3)

w4 = tf.get_variable(name="w4", shape=[512, 256],\
                          initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([256]))
L4 = tf.nn.relu(tf.matmul(L3,w4)+b4)

w5 = tf.get_variable(name="w5", shape=[256, 10],\
                          initializer = tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))

hf = tf.matmul(L4,w5)+b5
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hf, labels=y))

train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [18]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      myfeed = {x:batch_xs, y:batch_ys}
      cv, _ = sess.run([cost, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
    print("cost: {:.9f}".format(avg_cost))
    
  c_pre = tf.equal(tf.argmax(hf,1), tf.argmax(y,1))
  acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))
  print("acc: {}".format(sess.run(acc, feed_dict={x:mnist.test.images,y:mnist.test.labels})))

cost: 0.302022672
cost: 0.102606575
cost: 0.067886870
cost: 0.051074103
cost: 0.039023360
cost: 0.031265473
cost: 0.031841049
cost: 0.023885246
cost: 0.022906675
cost: 0.020802240
cost: 0.020642349
cost: 0.015204034
cost: 0.017492373
cost: 0.015104875
cost: 0.013471740
acc: 0.9768000245094299


# 지금 또 올려보는 건 쉽지 않다 : DropOut을 써보자

In [0]:
tf.reset_default_graph()
# graph에 있는 모든 tensor 초기화!

x = tf.placeholder(tf.float32, [None, 28*28])
y = tf.placeholder(tf.float32, [None, 10])

keep_prob = tf.placeholder(tf.float32)

w1 = tf.get_variable(name="w1", shape=[28*28, 512],\
                          initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(x,w1)+b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

w2 = tf.get_variable(name="w2", shape=[512, 512],\
                          initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1,w2)+b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

w3 = tf.get_variable(name="w3", shape=[512, 512],\
                          initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2,w3)+b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

w4 = tf.get_variable(name="w4", shape=[512, 256],\
                          initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([256]))
L4 = tf.nn.relu(tf.matmul(L3,w4)+b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

w5 = tf.get_variable(name="w5", shape=[256, 10],\
                          initializer = tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))

hf = tf.matmul(L4,w5)+b5
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hf, labels=y))

train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [20]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7}
      cv, _ = sess.run([cost, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
    print("cost: {:.9f}".format(avg_cost))
    
  c_pre = tf.equal(tf.argmax(hf,1), tf.argmax(y,1))
  acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))
  print("acc: {}".format(sess.run(acc, feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_prob:1})))

cost: 0.500910477
cost: 0.182771434
cost: 0.136398696
cost: 0.113254147
cost: 0.096473727
cost: 0.084625482
cost: 0.077895638
cost: 0.070867133
cost: 0.062645742
cost: 0.059896115
cost: 0.057252226
cost: 0.053166115
cost: 0.049032745
cost: 0.048748272
cost: 0.045094899
acc: 0.9804999828338623


In [0]:
# mnist model based

In [0]:
learning_rate = 0.001
training_epochs = 250
batch_size = 128

In [0]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None,28*28]) 
ximg = tf.reshape(x, [-1, 28, 28, 1]) # img batch*28*28*1(black/white)
# input(ximg) : batch , in_height, in_width, in_channels
y = tf.placeholder(tf.float32, [None,10])

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

w1 = tf.Variable(tf.random_normal([2,2,1,32]))
# filter(w) : filter_height, filter_width, in_channels, out_channels 형식으로 준비
L1 = tf.nn.conv2d(ximg, w1, strides=[1,1,1,1], padding='SAME') 
# 양 끝은 고정, 가운데 두개만 상하,좌우 / SAME 하면 같은 크기
# 요 conv2d를 하고 나면 batch * height * width * filter의 Tensor가 return됨
L1 = tf.nn.relu(L1)
# w1_1 = tf.Variable(tf.random_normal([2,2,32,32]))
# L1_1 = tf.nn.conv2d(L1, w1, strides=[1,1,1,1], padding='SAME') 
# L1_1 = tf.nn.relu(L1_1)
L1_1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
L1_1 = tf.nn.dropout(L1_1, keep_prob=keep_prob)
# L1 이미지 shape => (?, 28, 28, 1)
# conv => (?,28,28,32)
# relu => (?, 28, 28, 32)
# pooling => (?, 14, 14, 32)

w2 = tf.Variable(tf.random_normal([3,3,32,64]))
L2 = tf.nn.conv2d(L1_1, w2, strides=[1,1,1,1], padding='SAME') 
L2 = tf.nn.relu(L2)
# w2_1 = tf.Variable(tf.random_normal([3,3,64,64]))
# L2_1 = tf.nn.conv2d(L2, w2_1, strides=[1,1,1,1], padding='SAME') 
# L2_1 = tf.nn.relu(L2_1)
L2_1 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
L2_1 = tf.nn.dropout(L2_1, keep_prob=keep_prob)

# L2 이미지 shape => (?, 14, 14, 32)
# conv => (?,14,14,64)
# relu => (?, 14, 14, 64)
# pooling => (?, 7, 7, 64)
# reshape => (?, 7*7*64)

w3 = tf.Variable(tf.random_normal([3,3,64,128]))
L3 = tf.nn.conv2d(L2_1, w3, strides=[1,1,1,1], padding='SAME') 
L3 = tf.nn.relu(L3)
# w3_1 = tf.Variable(tf.random_normal([3,3,128,128]))
# L3_1 = tf.nn.conv2d(L3, w3_1, strides=[1,1,1,1], padding='SAME') 
# L3_1 = tf.nn.relu(L3_1)
L3_1 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
L3_1 = tf.nn.dropout(L3_1, keep_prob=keep_prob)

L3_flat = tf.reshape(L3_1, [-1,4*4*128])

w4 = tf.get_variable("w4", shape=[4*4*128, 625], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([625]))
L4 = tf.matmul(L3_flat, w4) + b1
L4 = tf.nn.dropout(L4, keep_prob=keep_prob_flatten)

w5 = tf.get_variable("w5", shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4, w5) + b2
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten) 

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))
train = tf.train.RMSPropOptimizer(learning_rate, 0.9).minimize(cost)

In [104]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.7}
      cv, _ = sess.run([cost,train], feed_dict=myfeed)
      avg_cost += cv/total_batch
    accv = sess.run(acc, feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1,keep_prob_flatten:1})
    print("cost: {:.9f}, test_acc: {}".format(avg_cost, accv))

  print("final_acc: {}".format(sess.run(acc, feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1,keep_prob_flatten:1})))

cost: 701.612975203, test_acc: 0.8959000110626221
cost: 16.019158299, test_acc: 0.8398000001907349
cost: 3.081187980, test_acc: 0.7818999886512756
cost: 1.831232527, test_acc: 0.7318999767303467
cost: 1.517682626, test_acc: 0.7272999882698059
cost: 1.351400374, test_acc: 0.8341000080108643
cost: 1.229608498, test_acc: 0.853600025177002
cost: 1.142247755, test_acc: 0.9020000100135803
cost: 1.066612816, test_acc: 0.9081000089645386
cost: 1.007258495, test_acc: 0.9302999973297119
cost: 0.961867248, test_acc: 0.9362000226974487
cost: 0.914491587, test_acc: 0.9386000037193298
cost: 0.882469982, test_acc: 0.9470000267028809
cost: 0.845083625, test_acc: 0.9476000070571899
cost: 0.817048594, test_acc: 0.9380000233650208
cost: 0.783716107, test_acc: 0.9531999826431274
cost: 0.760245128, test_acc: 0.9624999761581421
cost: 0.742029335, test_acc: 0.9592999815940857
cost: 0.720276666, test_acc: 0.9581999778747559
cost: 0.693798021, test_acc: 0.9652000069618225
cost: 0.681780291, test_acc: 0.9677000

In [47]:
ximg

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [36]:
mnist.test.labels.shape

(10000, 10)